In [1]:
cd ..

/Users/lisawink/Documents/freiburg


# Relationships across scales

In [2]:
import pandas as pd
import geopandas as gpd
from calc_params import calculate_statistics
from tqdm import tqdm
import pandas as pd
from tqdm import tqdm
import numpy as np
from scipy.stats import pearsonr, spearmanr

In [3]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
# Set default font globally
pio.templates["custom"] = pio.templates["plotly"]  # Use a base template like "plotly_white"
pio.templates["custom"].layout.font = dict(
    size=16,  # Default font size
)
# Make the custom template the default
pio.templates.default = "custom"

"heat island day" is defined as nights were difference between rural and urban is > e.g. 4 deg + no precipitation

In [4]:
prec23 = pd.read_csv('/Users/lisawink/Documents/paper1/data/raw_data/FRCHEM_2023_Precipitation_Daily_UTC.csv')
prec22 = pd.read_csv('/Users/lisawink/Documents/paper1/data/raw_data/FRCHEM_2022_Precipitation_Daily_UTC.csv')
# concat prec22 and prec23
prec = pd.concat([prec22, prec23])
prec['datetime_UTC'] = pd.to_datetime(prec['YYYYMMDDHHMM_From'], format='%Y%m%d%H%M', utc=True)

In [5]:
fig = px.line(prec, x='datetime_UTC', y='Precipitation_Sum_mm')
fig.show()

In [6]:
temp =  pd.read_csv('/Users/lisawink/Documents/paper1/data/gap_filled_data_ta_rh.csv')
temp['datetime_UTC']=pd.to_datetime(temp['datetime_UTC'])
temp = temp[temp['variable']=='Ta_deg_C']
FRHOCH = temp[temp['station_id'] == 'FRHOCH']
FRKART = temp[temp['station_id'] == 'FRKART']
FRTEMP = pd.merge(FRHOCH, FRKART, on='datetime_UTC', how='inner', suffixes=('_HOCH', '_KART'))
FRTEMP['difference'] = FRTEMP['value_KART'] - FRTEMP['value_HOCH']
heat_island_dates = FRTEMP[(FRTEMP['difference'] > 4)]['datetime_UTC']
no_prec = prec[(prec['datetime_UTC'].dt.date.isin(heat_island_dates.dt.date)) & (prec['Precipitation_Sum_mm'] == 0)]['datetime_UTC'].dt.date
heat_island_dates = heat_island_dates[heat_island_dates.dt.date.isin(no_prec)]

Choose only nighttime (3,4,5am) and winter months (Nov-Feb)

In [7]:
heat_island_winter_nights = heat_island_dates[((heat_island_dates.dt.hour == 3) |  (heat_island_dates.dt.hour == 4) | (heat_island_dates.dt.hour == 5)) & (heat_island_dates.dt.month.isin([1, 2, 3, 11, 12]))]

In [30]:
heat_island_winter_nights

1491   2022-11-02 03:00:00+00:00
1492   2022-11-02 04:00:00+00:00
1635   2022-11-08 03:00:00+00:00
1636   2022-11-08 04:00:00+00:00
1637   2022-11-08 05:00:00+00:00
1707   2022-11-11 03:00:00+00:00
1708   2022-11-11 04:00:00+00:00
1709   2022-11-11 05:00:00+00:00
2643   2022-12-20 03:00:00+00:00
2644   2022-12-20 04:00:00+00:00
2645   2022-12-20 05:00:00+00:00
3867   2023-02-09 03:00:00+00:00
3868   2023-02-09 04:00:00+00:00
3891   2023-02-10 03:00:00+00:00
3893   2023-02-10 05:00:00+00:00
3916   2023-02-11 04:00:00+00:00
3917   2023-02-11 05:00:00+00:00
3940   2023-02-12 04:00:00+00:00
3941   2023-02-12 05:00:00+00:00
3963   2023-02-13 03:00:00+00:00
3964   2023-02-13 04:00:00+00:00
3965   2023-02-13 05:00:00+00:00
4035   2023-02-16 03:00:00+00:00
4036   2023-02-16 04:00:00+00:00
4037   2023-02-16 05:00:00+00:00
4133   2023-02-20 05:00:00+00:00
4155   2023-02-21 03:00:00+00:00
4156   2023-02-21 04:00:00+00:00
4157   2023-02-21 05:00:00+00:00
4395   2023-03-03 03:00:00+00:00
4396   202

In [8]:
temp['datetime_UTC'] = temp['datetime_UTC'].astype(str)
temp = temp.pivot(index='station_id', columns='datetime_UTC', values='value')

# Statistics for top nighttime parameters only
Top nighttime parameters defined as the parameters with the 100 highest mean mutual information with temperatures on heat island days at 4am across all scales (as calculated in stratify.ipynb)

In [10]:
top_params = pd.read_csv('/Users/lisawink/Documents/paper1/data/processed_data/top_params.csv')

In [11]:
top_params = list(top_params.values.flatten())

When running for first time

In [ ]:
for time in tqdm(heat_island_winter_nights):
    print(time)
    
    radii = [30,40,50,60,70,80,90,100,120,140,160,180,200,250,300,400,500,750]
    stats_dict = {}
    for i in tqdm(radii):
        params = gpd.read_parquet(f'/Users/lisawink/Documents/paper1/data/processed_data/processed_station_params_{i}.parquet')
        params = params.set_index('station_id')
        if params.empty:
            continue
        
        to_remove = ['station_no','station_name','station_long_name','station_type','station_lat','station_lon','station_elevation','mounting_structure','sky_view_factor','dominant_land_use','local_climate_zone','urban_atlas_class','urban_atlas_code','geometry','SVF']
        params = params.drop(to_remove, axis=1)
        params = params[top_params]
        params = params.merge(temp[str(time)], left_on='station_id', right_on='station_id',how='inner')
        stats = calculate_statistics(params, str(time))
        stats.index = stats['Parameter']
        stats.to_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/processed_station_stats_top_params{i,time}.csv', index=False)
        stats_dict[i] = stats

In [9]:
def _ci_index(data):
    lower = np.percentile(data, 2.5)
    upper = np.percentile(data, 97.5)

    index = (upper - lower) / (upper + lower)/2
    return index

def bootstrap(func, params, x_col, y_col, n_bootstrap=1000):

    X = params.loc[:,[x_col]].values.ravel()  # Your first variable
    Y = params.loc[:,[y_col]].values.ravel()
    # Step 1: Calculate observed  correlation
    rho_obs, _ = func(X, Y)

    # Step 2: Bootstrap  correlation
    bootstrap_corr = []

    for _ in range(n_bootstrap):
        # Resample data with replacement
        indices = np.random.choice(len(X), len(X), replace=True)
        X_bootstrap = X[indices]
        Y_bootstrap = Y[indices]
        rho_boot, _ = func(X_bootstrap, Y_bootstrap)
        bootstrap_corr.append(rho_boot)

    # Step 3: Null distribution by permuting Y
    n_null = n_bootstrap
    null_corr = []

    for _ in range(n_null):
        Y_permuted = np.random.permutation(Y)  # Shuffle Y
        rho_null, _ = func(X, Y_permuted)
        null_corr.append(rho_null)

    # Step 4: Calculate p-values
    bootstrap_corr = np.array(bootstrap_corr)
    null_corr = np.array(null_corr)

    # Bootstrap p-value
    p_bootstrap = np.mean(bootstrap_corr >= rho_obs)

    # Null p-value
    p_null = np.mean(null_corr >= rho_obs)

    # Output results
    results = {
        "Observed correlation": rho_obs,
        "Mean correlation (bootstrap)": np.mean(bootstrap_corr),
        "Standard deviation (bootstrap)": np.std(bootstrap_corr),
        "95% confidence interval (bootstrap)": np.percentile(bootstrap_corr, [2.5, 97.5]),
        "95% confidence interval index (bootstrap)": _ci_index(bootstrap_corr),
        "P-value (bootstrap)": p_bootstrap,
        "P-value (null)": p_null
    }

    return results

In [10]:
def bootstrap_param(var):
    for time in heat_island_winter_nights:
        radii = [30,40,50,60,70,80,90,100,120,140,160,180,200,250,300,400,500,750]
        stats_dict = {}
        for i in radii:
            params = gpd.read_parquet(f'/Users/lisawink/Documents/paper1/data/processed_data/processed_station_params_{i}.parquet')
            params = params.set_index('station_id')
            if params.empty:
                continue
            to_remove = ['station_no','station_name','station_long_name','station_type','station_lat','station_lon','station_elevation','mounting_structure','sky_view_factor','dominant_land_use','local_climate_zone','urban_atlas_class','urban_atlas_code','geometry','SVF']
            params = params.drop(to_remove, axis=1)
            params = params[[var]]
            params = params.dropna()
            params = params.merge(temp[str(time)], left_on='station_id', right_on='station_id',how='inner')
            stats = bootstrap(spearmanr, params, var,str(time), n_bootstrap=1000)
            stats_df = pd.DataFrame(stats)  # Convert stats to DataFrame
            stats_df.to_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/bootstrap/bootstrap_{var,i,time}.csv', index=False)

def bootstrap_plot(var):
    radii = [30,40,50,60,70,80,90,100,120,140,160,180,200,250,300,400,500,750]
    mean_stats = {}
    fig = go.Figure()

    # Create a colormap
    colormap = px.colors.sample_colorscale('Phase', [i/11 for i in range(12)])

    # Extract the month from the datetime format and normalize
    normalize = lambda x: (x - 1) / 11

    for time in heat_island_winter_nights:
        hw_list = []
        for i in radii:
            stats = pd.read_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/bootstrap/bootstrap_{var,i,time}.csv')
            hw_list.extend(stats['Standard deviation (bootstrap)'])
        month = pd.to_datetime(time).month
        color = colormap[int(normalize(month) * 11)]
        fig.add_trace(go.Scatter(x=radii, y=hw_list, mode='lines', name=str(time), line=dict(color=color)))
    fig.update_layout(
        title = var + " Bootstrap Standard Deviation of Spearman Correlation Coefficient",
        xaxis_title = "Radii",
        yaxis_title = "Bootstrap Standard Deviation",
    )
    fig.show()

def param_scale_plot(var):
    radii = [30,40,50,60,70,80,90,100,120,140,160,180,200,250,300,400,500,750]
    mean_stats = {}
    fig = go.Figure()

    # Create a colormap
    colormap = px.colors.sample_colorscale('Phase', [i/11 for i in range(12)])

    # Extract the month from the datetime format and normalize
    normalize = lambda x: (x - 1) / 11

    for time in heat_island_winter_nights:
        hw_list = []
        for i in radii:
            stats = pd.read_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/processed_station_stats_top_params{i,time}.csv')
            hw_list.extend(stats[stats['Parameter'] == var]['Spearman Correlation'].values)
        month = pd.to_datetime(time).month
        color = colormap[int(normalize(month) * 11)]
        fig.add_trace(go.Scatter(x=radii, y=hw_list, mode='lines', name=str(time), line=dict(color=color)))
    fig.update_layout(
        title="Spearman Correlation of " + var + " Against Heat Island Nighttime Temperature Across Scales",
        xaxis_title="Radii",
        yaxis_title="Spearman Correlation",
    )
    fig.show()

# Urban Canyon (Distribution)

## HW Ratio

### Median

In [19]:
param_scale_plot('StrHW_median')

In [28]:
bootstrap_plot('StrHW_median')

### StrHW_max

In [30]:
param_scale_plot('StrHW_max')

In [51]:
bootstrap_plot('StrHW_max')

## Street Openess

In [13]:
param_scale_plot('StrOpe_median')

In [ ]:
bootstrap_param('StrOpe_median')

In [15]:
bootstrap_plot('StrOpe_median')

## Height

### Mean

In [52]:
param_scale_plot('BuHt_mean')

In [38]:
bootstrap_param('BuHt_mean')

In [53]:
bootstrap_plot('BuHt_mean')

### Area weighted mean

### BuHt_wmean

In [36]:
param_scale_plot('BuHt_wmean')

In [35]:
bootstrap_param('BuHt_wmean')
bootstrap_plot('BuHt_wmean')

### BuHt_max

In [54]:
param_scale_plot('BuHt_max')

In [37]:
bootstrap_param('BuHt_max')

In [55]:
bootstrap_plot('BuHt_max')

# Building Magnitude (Dimension)

## Area

### BuAre_sum

In [56]:
param_scale_plot('BuAre_sum')

In [16]:
bootstrap_param('BuAre_sum')

In [17]:
bootstrap_plot('BuAre_sum')

## Surface Area

### BuSA_3D_sum

In [58]:
param_scale_plot('BuSA_3D_sum')

In [40]:
bootstrap_param('BuSA_3D_sum')

In [18]:
bootstrap_plot('BuSA_3D_sum')

### BuSA_3D_max

In [60]:
param_scale_plot('BuSA_3D_max')

In [39]:
bootstrap_param('BuSA_3D_max')


In [61]:
bootstrap_plot('BuSA_3D_max')

## Volume

### BuVol_3D_sum

In [62]:
param_scale_plot('BuVol_3D_sum')

In [41]:
bootstrap_param('BuVol_3D_sum')

In [63]:
bootstrap_plot('BuVol_3D_sum')

### BuVol_3D_max

In [64]:
param_scale_plot('BuVol_3D_max')

In [42]:
bootstrap_param('BuVol_3D_max')

In [19]:
bootstrap_plot('BuVol_3D_max')

# Building Complexity (Shape)

## Fractal Form

### BuFF_mean

In [66]:
param_scale_plot('BuFF_mean')

In [43]:
bootstrap_param('BuFF_mean')

In [67]:
bootstrap_plot('BuFF_mean')

### BuFF_wmean

In [68]:
param_scale_plot('BuFF_wmean')

In [44]:
bootstrap_param('BuFF_wmean')

In [69]:
bootstrap_plot('BuFF_wmean')

### BuFF_wmedian

In [70]:
param_scale_plot('BuFF_wmedian')

In [81]:
bootstrap_param('BuFF_wmedian')

In [20]:
bootstrap_plot('BuFF_wmedian')

### BuFra_3D_median

In [21]:
param_scale_plot('BuFra_3D_median')

In [22]:
bootstrap_param('BuFra_3D_median')

In [23]:
bootstrap_plot('BuFra_3D_median')

## Square Compactness

In [24]:
param_scale_plot('BuSqC_wmean')

In [25]:
bootstrap_param('BuSqC_wmean')

In [26]:
bootstrap_plot('BuSqC_wmean')

## Elongation

In [27]:
param_scale_plot('BuElo_median')

In [28]:
bootstrap_param('BuElo_median')

In [29]:
bootstrap_plot('BuElo_median')

# Compactness (Distribution)

## Shared Walls Ratio

### BuSWR_median

In [71]:
param_scale_plot('BuSWR_median')

In [45]:
bootstrap_param('BuSWR_median')

In [72]:
bootstrap_plot('BuSWR_median')

### BuSWR_3D_median

In [73]:
param_scale_plot('BuSWR_3D_median')

In [46]:
bootstrap_param('BuSWR_3D_median')

In [74]:
bootstrap_plot('BuSWR_3D_median')

### BuSWR_3D_max

In [75]:
param_scale_plot('BuSWR_3D_max')

In [47]:
bootstrap_param('BuSWR_3D_max')

In [76]:
bootstrap_plot('BuSWR_3D_max')

## Adjacency

In [77]:
param_scale_plot('BuAdj')

### 

In [ ]:
bootstrap_param('BuAdj')

In [78]:
bootstrap_plot('BuAdj')

## Inter-Building Distance

In [79]:
param_scale_plot('BuIBD')

In [ ]:
bootstrap_param('BuIBD')

In [80]:
bootstrap_plot('BuIBD')